<a href="https://colab.research.google.com/github/nam4dev/pandas_introduction_presentation/blob/master/pandas_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import builtins
import pathlib as path
# Import requests library
import requests
# Import numpy library
import numpy as np
# Import pandas library
import pandas as pd
# Import matplotlib library
import matplotlib.pyplot as plt


In [0]:
filename = './cell_phones.csv'
cell_phones_file = path.Path(filename)

if not cell_phones_file.exists():
  # Getting datasets from https://www.gapminder.org/data/ in CSV format
  response = requests.get(
      'https://docs.google.com/spreadsheet/pub?'
      'key=0AkBd6lyS3EmpdEhWLWtqNzljbWg4ZXV6M09JQXNGaUE&output=csv'
  )
  # Writing the response to the filesystem
  cell_phones_file.open('w').write(response.text)

In [0]:
#@title Display data content { run: "auto", vertical-output: true }
display = False #@param {type:"boolean"}

if display:
    # Read './cell_phones.csv'
    print(cell_phones_file.open().read())

In [0]:
# Setting Pandas general options
pd.set_option('display.max_rows', 500)

In [0]:
# Loading the data into Pandas dataframe using 
# 'Mobile cellular subscriptions, total number' as index column
df_cell_phones = pd.read_csv(
    'cell_phones.csv', 
    index_col='Mobile cellular subscriptions, total number'
)

In [0]:
#@title Row number { run: "auto", vertical-output: true }
n_head = 5 #@param {type:"slider", min:1, max:100, step:1}

# Print data frame head (first rows)
df_cell_phones.head(n_head)

In [0]:
#@title Row number { run: "auto", vertical-output: true }
n_tail = 5 #@param {type:"slider", min:1, max:100, step:1}

# Print data frame tail (last rows)
df_cell_phones.tail(n_tail)

In [0]:
# Display data frame shape
df_cell_phones.shape

In [0]:
# Print data frame info
df_cell_phones.info()

In [0]:
# Display data frame statistics
df_cell_phones.describe()

In [0]:
# Selecting the 2011 data for all countries (by position)
df_cell_phones_2011 = df_cell_phones.iloc[:, 46]
df_cell_phones_2011.head()

In [0]:
# Selecting the 2011 data for all countries (by index)
df_cell_phones_2011 = df_cell_phones.loc[:, '2011']
df_cell_phones_2011.head()

In [0]:
# Selecting the 2000ies data for all countries (by index)
years_range = [str(year) for year in range(2000, 2012)]
print('years_range =>', years_range)
df_cell_phones_2000_to_2011 = df_cell_phones[years_range]
df_cell_phones_2000_to_2011.head()

In [0]:
# Renaming 'Mobile cellular subscriptions, total number' to 'Country'
df_cell_phones.index.names = ['Country']
# Print the head of the data frame
df_cell_phones.head()

In [0]:
## Clean data frame ##

# Convert zero values to "not a number" entries
new_df = df_cell_phones.replace(0, np.nan)
# Drop all year where all cells are `np.nan` entries
new_df = new_df.dropna(axis=1, how='all')
# Drop all countries where all cells are `np.nan` entries
new_df = new_df.dropna(how='all')
# Fill all remaining `np.nan` entries with 0.0
new_df.fillna(0, inplace=True)

new_df.head()

In [0]:
all_countries = set(df_cell_phones.index)
# Countries without known subscriptions
countries = sorted([c.strip() for c in all_countries - set(new_df.index)])
print('Countries without (known) subscriptions:')
for c in countries:
    print('\t- {}'.format(c))


In [0]:
# Getting more than 500,000,000 subscriptions countries for year 2011
# Create the boolean array: high_subscriptions
year = '2011'
subscriptions_amount = 500000000
high_subscriptions = new_df[year] > subscriptions_amount

# Filter the cell phones DataFrame with 
# the high_subscriptions array: high_subscriptions_df
high_subscriptions_df = new_df[high_subscriptions]

# Print the high_subscriptions_df DataFrame
high_subscriptions_df

In [0]:
# Print countries
print('Country with more than {} subscriptions '
      'for {}:'.format(subscriptions_amount, year))
for c in high_subscriptions_df.index:
    print('\t- {} => {}'.format(c, high_subscriptions_df.loc[c, year]))


In [0]:
# Sum up subscriptions by year
# sum_by_years is of type `pandas.core.series.Series`
sum_by_years = new_df.sum(axis=0) # or axis='rows'
sum_by_years.head()

In [0]:
# Sum up subscriptions by country
# sum_by_coutries is of type `pandas.core.series.Series`
sum_by_coutries = new_df.sum(axis=1)  # or axis='columns'
sum_by_coutries.head()

In [0]:
%matplotlib inline

ymax = sum_by_years.max()
print('Maximum subscriptions by years is' 
      ' {} => {}'.format(sum_by_years.idxmax(), ymax))
sum_by_years.plot(color='blue')

plt.xlabel('Year')
plt.ylabel('Cellphone subscriptions')
plt.title('Progression of cumulative subscriptions by year')

plt.show()

In [0]:
# Compute min & max info (key, value)
cmax, cmin = sum_by_coutries.max(), sum_by_coutries.min()
smin = list(sum_by_coutries.index).index(sum_by_coutries.idxmin())
smax = list(sum_by_coutries.index).index(sum_by_coutries.idxmax())

print('Maximum subscriptions by countries is'
      ' {} => {}'.format(sum_by_coutries.idxmax(), cmax))

# Plot the series
sum_by_coutries.plot.bar(figsize=(35, 5), color='orange')

# Set X, Y lables & Plot title
plt.xlabel('Country')
plt.ylabel('Cellphone subscriptions')
plt.title('Progression of cumulative subscriptions by countries')

# Minimum Annotation
plt.annotate(
    'Minimum ({})'.format(cmin), 
    xy=(smin, cmin), 
    xytext=(smin, 3e9), 
    arrowprops=dict(facecolor='blue')
)
# Maximum Annotation
plt.annotate(
    'Maximum ({})'.format(cmax), 
    xy=(smax, cmax), 
    xytext=(smax + 5, cmax), 
    arrowprops=dict(facecolor='green')
)

# Mauritius annotation
mauritius = 'Mauritius'
cmauritius = sum_by_coutries[mauritius]
smauritius = list(sum_by_coutries.index).index(mauritius)
plt.annotate(
    'Mauritius ({})'.format(cmauritius), 
    xy=(smauritius, cmauritius), 
    xytext=(smauritius, 2e9), 
    arrowprops=dict(facecolor='purple')
)

# Show the plot
plt.show()

In [0]:
# Saving transformed data (new_df) to JSON
filename = './cell_phones.json'
cell_phones_json = path.Path(filename)
new_df.to_json(cell_phones_json.as_posix())

cell_phones_json.open().read()